# Error handling

Error handling in Rust is very different from both C and C++. The Rust's approach to error handling has advantages and downsides to both, and it worth a chapter to cover.

## Error handling in C

C doesn't have exceptions, try/catch and stack unwinding. It uses return values for declaring that something bad happened, and global variables if more information is needed. The similar is possible is Rust, but we (and compiler) don't like global mutable variables since they are not thread safe. Even if they were thread safe (we can use thread local variables for example), since global variables are ugly, and Rust has ADT, we can use an enum to store error data inside the return value:

In [3]:
enum ReturnTypeOfAFunction {
    EverythingIsFine {
        result: i32,
    },
    ConnectionFailure {
        url: String,
    },
    DiskError {
        file_name: String,
    },
    UnknownError,
}

This pattern is not specific to this function, so the standard library has `enum Result<T, E>` with variants `Ok(T)` and `Err(E)`. We can make the above type using it:

In [ ]:
enum ErrorsOfAFunction {
    ConnectionFailure {
        url: String,
    },
    DiskError {
        file_name: String,
    },
    UnknownError,
}

// type alias
type ReturnTypeOfAFunction = Result<i32, ErrorsOfAFunction>;

The Rust standard library and ecosystem widely use the `Result` enum for error handling:

In [5]:
use std::fs;

fs::read_to_string("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

With `Result`, try/catch is just a `match`:

In [14]:
let open_result = fs::read_to_string("non-existent.txt");
match open_result {
    Ok(file_string) => {
        println!("File content is: {}", file_string);
    }
    Err(error) => {
        println!("Some error happened: {}", error);
    }
}
println!("Other codes");

Some error happened: No such file or directory (os error 2)
Other codes


## ? Operator

Now think we are in a function, and want to cascade error. We can use this code:

In [18]:
fn find_count_of_a_in_a_file(path: &str) -> Result<usize, std::io::Error> {
    let open_result = fs::read_to_string("non-existent.txt");
    let text = match open_result {
        Ok(file_string) => file_string,
        Err(error) => return Err(error),
    };
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

Since cascading error is a too common thing, Rust has an operator for it, `?` operator. `expr?` means match `expr` and in case of `Err`, return the error from current function, and in case of `Ok`, evaluate `expr?` to the inner thing of `Ok`. Example:

In [19]:
fn find_count_of_a_in_a_file(path: &str) -> Result<usize, std::io::Error> {
    let text = fs::read_to_string("non-existent.txt")?;
                                                // --^-- notice this
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

You can't use `?` on results in functions that doesn't return result:

In [20]:
fn find_count_of_a_in_a_file_wrong(path: &str) -> usize {
    let text = fs::read_to_string("non-existent.txt")?;
    let result = text.chars().filter(|x| *x == 'a').count();
    result
}

Error: the `?` operator can only be used in a function that returns `Result` or `Option` (or another type that implements `FromResidual`)

Compiler spoiled the story. As compiler said, `?` also works in functions that return `Option<T>`, for things with type `Option<U>`:

In [21]:
fn third_from_an_optional_vector(input_opt: Option<Vec<i32>>) -> Option<i32> {
    let input = input_opt?; // return None if input was None
    let third_opt = input.get(2); // Similar to input[2], but returns Option
    let third = third_opt?; // Return None if there were no third element. You could do it in one line.
    Some(*third)
}

println!("{:?}", third_from_an_optional_vector(None));
println!("{:?}", third_from_an_optional_vector(Some(vec![1, 2, 3, 4])));
println!("{:?}", third_from_an_optional_vector(Some(vec![1])));

None
Some(3)
None


You can convert a result to an option with `.ok()`

In [22]:
let x: Result<i32, String> = Ok(12);
x.ok()

Some(12)

And an option to a result with `.ok_or`:

In [24]:
let x: Option<i32> = None;
x.ok_or("some_error")

Err("some_error")

So we can use `?` for `Option` in a function that returns `Result` and wise versa.

In [28]:
use std::io::Error;

#[derive(Debug)]
enum MyError {
    IoError(Error),
    PathNotProvided
}

impl From<Error> for MyError {
    fn from(e: Error) -> MyError {
        MyError::IoError(e)
    }
}

fn find_count_of_a_in_a_file(path: Option<&str>) -> Result<usize, MyError> {
    let path = path.ok_or(MyError::PathNotProvided)?;
    let text = fs::read_to_string("non-existent.txt")?;
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file(None)

Err(PathNotProvided)

As you can see, we can use `?` for different error types if they implement `From` trait.

## Control flow enum

There is another enum which supports `?` operator: `ControlFlow<B, C = ()>` with variants `Break(B)` and `Continue(C)`. For example, have this code:

In [29]:
for x in 1..20 {
    if x % 2 == 0 {
        println!("{x} 2");
        if x % 3 == 0 {
            println!("{x} 3");
            if x % 4 == 0 {
                println!("{x} 4");
                break;
            }
        }
    }
}

2 2
4 2
6 2
6 3
8 2
10 2
12 2
12 3
12 4


()

Now we want to refactor two inner ifs as a separate function. It doesn't work naively:

In [32]:
fn refactored(x: i32) {
    println!("{x} 2");
    if x % 3 == 0 {
        println!("{x} 3");
        if x % 4 == 0 {
            println!("{x} 4");
            break;
        }
    }
}

for x in 1..20 {
    if x % 2 == 0 {
        refactored(x);
    }
}

Error: `break` outside of a loop

We can use `ControlFlow` enum as a glorified `bool`:

In [33]:
use std::ops::ControlFlow;

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        println!("{x} 3");
        if x % 4 == 0 {
            println!("{x} 4");
            return ControlFlow::Break(());
        }
    }
    ControlFlow::Continue(())
}

for x in 1..20 {
    if x % 2 == 0 {
        if refactored(x).is_break() {
            break;
        }
    }
}

2 2
4 2
6 2
6 3
8 2
10 2
12 2
12 3
12 4


()

Now Imagine we want to refactor it one more level. Without `?` it would look like this:

In [34]:
fn refactored2(x: i32) -> ControlFlow<()> {
    println!("{x} 3");
    if x % 4 == 0 {
        println!("{x} 4");
        return ControlFlow::Break(());
    }
    ControlFlow::Continue(())
}

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        if refactored2(x).is_break() {
            return ControlFlow::Break(());
        }
    }
    ControlFlow::Continue(())
}

But with `?`, second refactoring becomes almost copy pasting the code into a new function:

In [ ]:
fn refactored2(x: i32) -> ControlFlow<()> {
    println!("{x} 3");
    if x % 4 == 0 {
        println!("{x} 4");
        return ControlFlow::Break(());
    }
    ControlFlow::Continue(())
}

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        refactored2(x)?;
    }
    ControlFlow::Continue(())
}

Another usage of `ControlFlow` enum is in traversing trees (and graphs and anything). Let's define a simple tree data structure:

In [36]:
#[derive(Debug)]
enum Tree<T> {
    Leaf(T),
    Node(Vec<Tree<T>>), // vector of children
}

use Tree::*;

let example = Node(vec![Leaf(10), Node(vec![Leaf(20), Leaf(30), Leaf(40)]), Leaf(50)]);
println!("{:#?}", example);

Node(
    [
        Leaf(
            10,
        ),
        Node(
            [
                Leaf(
                    20,
                ),
                Leaf(
                    30,
                ),
                Leaf(
                    40,
                ),
            ],
        ),
        Leaf(
            50,
        ),
    ],
)


Now we want to find left most node which is bigger than a number:

In [43]:
fn left_most_bigger<T: Ord + Copy>(tree: &Tree<T>, than_this: T) -> Option<T> {
    match tree {
        Leaf(x) => (x > &than_this).then_some(*x),
        Node(children) => {
            for child in children {
                match left_most_bigger(&child, than_this) {
                    Some(x) => return Some(x),
                    None => continue,
                }
            }
            None
        }
    }
}

left_most_bigger(&example, 35)

Some(40)

Now let's change the `Option<T>` to `ControlFlow<T>`:

In [46]:
fn left_most_bigger<T: Ord + Copy>(tree: &Tree<T>, than_this: T) -> ControlFlow<T> {
    match tree {
        Leaf(x) if x > &than_this => ControlFlow::Break(*x),
        Leaf(x) => ControlFlow::Continue(()),
        Node(children) => {
            for child in children {
                left_most_bigger(&child, than_this)?;
            }
            ControlFlow::Continue(())
        }
    }
}

left_most_bigger(&example, 35)

Break(40)

We can use `?` and break from the outer function if we founded what we need. `ControlFlow` has reverse `?` semantic to `Option` and `Result`, and it will return in case of success. `break` with value is also supported in `loop`: 

In [47]:
let mut counter = 0;

let result = loop {
    counter += 1;

    if counter == 10 {
        break counter * 2;
    }
};

result

20

That's enough about control flow. Let's get back to error handling.

Error handling with `Result` has these benefits:
* Possible errors (or lack of error) of a function is clear from it's return type.
* Error handling is explicit. You can't forget that a function may throw error. You need to cascade it yourself with `?` explicitly.
* No need to support runtime for stack unwinding. This is an issue in low level system programming, like in OS or embedded systems. People using C++ in those use-cases use `-fno-exception`, but it will make any error in standard library and third party libraries impossible to catch and handle.

## Panic

There is another form of errors that might happen in a rust program:

In [48]:
:preserve_vars_on_panic 1

Preserve vars on panic: true


In [49]:
let v = vec![1, 2, 3];
v[5]

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 5', src/lib.rs:213:40
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic_bounds_check
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:84:5
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: run_user_code_24
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Panic occurred, the following variables have been lost: v

You can panic manually, using a macro similar to `println!`:

In [50]:
fn some_panicking_fn(x: i32) {
    panic!("I don't like {x}");
}

some_panicking_fn(32);

thread '<unnamed>' panicked at 'I don't like 32', src/lib.rs:29:5
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: <unknown>
   3: <unknown>
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Outside of jupyter notebook, it shows a meaningful stacktrace so you can see why some panic happened.

Panic is for the case that something wrong happens. That means, each panic in the production means there exists some bug in the code, and user ideally should never see a panic. One use case of panic is during development. For example, Rust doesn't allow functions to not return anything, since it will result in uninitialized memory and is UB:

In [51]:
fn some_fn_that_returns_something_complex() -> std::fs::File {
    println!("some initial works");
}

Error: mismatched types

`todo!()` is a function that panics, and is useful for silencing compiler in these cases:

In [53]:
use std::fs::File;

fn some_fn_that_returns_something_complex() -> File {
    println!("some initial works");
    todo!()
}

let some_file: File = some_fn_that_returns_something_complex();

some initial works


thread '<unnamed>' panicked at 'not yet implemented', src/lib.rs:56:5
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:48:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Panic occurred, the following variables have been lost: some_file

Another use case of panic is when compiler can't understand some code path is not reachable. In those cases, we can use `unreachable!()`:

In [55]:
fn collatz(x: i32) -> i32 {
    match x % 2 {
        0 => x / 2,
        1 => 3 * x + 1,
        // Compiler doesn't know that `x % 2` is always 0 or 1
        _ => unreachable!(),
    }
}

collatz(13)

40

If our assumption about `unreachable!()`ity becomes wrong (which is wrong in the above example) and we reach that path, we will get a panic:

In [56]:
collatz(-3)

thread '<unnamed>' panicked at 'internal error: entered unreachable code', src/lib.rs:11:14
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:48:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


We can convert `Result<T,E>` and `Option<T>` to their inner value `T`, and panic on unhappy case, using `.unwrap()`:

In [57]:
let x_result: Result<i32, &'static str> = Err("some error");
let x: i32 = x_result.unwrap();

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: "some error"', src/lib.rs:216:23
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::result::unwrap_failed
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/result.rs:1805:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Panic occurred, the following variables have been lost: x_result, x

In library code, panic doesn't necessarily means a bug. It means either a bug, or a violation of function's preconditions. For example, `.unwrap()` or array indexing in the standard library deliberately panics, because they assume that caller guarantees `Option` is not `None` or array index is not out of bound. We can use this in our functions as well:

In [62]:
fn is_prime(i: i32) -> bool {
    if i < 2 {
        return false;
    }
    !(2..i).take_while(|x| x * x <= i).any(|x| i % x == 0)
}

fn primes_until_this(i: i32) -> usize {
    assert!(i >= 0); // this function not really mean for non positive numbers
    // without this assert, we will return a garbage 0
    (0..=i).filter(|x| is_prime(*x)).count()
}

primes_until_this(10)

4

In [63]:
primes_until_this(-10);

thread '<unnamed>' panicked at 'assertion failed: i >= 0', src/lib.rs:79:5
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:48:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Libraries might want to expose to variants for a function, one panicking, and one returning `Option` or `Result`. For example, `v[i]` panics and `v.get(i)` returns an `Option`. Although most of times the returning variant is sufficient, since panic is one `.unwrap()` away. But when a function is used a lot (like indexing), `.unwrap()` might becomes too wordy.

## What happens at panicking

Panic implementation is decided by the main binary, so libraries can only depend on it being some function that won't return. By default, it will unwind the stack and finishes the current thread. Other threads can see if that thread panicked from `JoinHandle`:

In [64]:
(1..10).map(|x| {
    let join_handle = std::thread::spawn(move || {
        assert!(is_prime(x)); // some test code
    });
    let join_result = join_handle.join();
    (x, join_result)
}).collect::<Vec<_>>()

thread '<unnamed>' panicked at 'assertion failed: is_prime(x)', src/lib.rs:244:9
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:48:5
   3: <unknown>
   4: <unknown>
   5: <alloc::boxed::Box<F,A> as core::ops::function::FnOnce<Args>>::call_once
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/alloc/src/boxed.rs:1951:9
   6: <alloc::boxed::Box<F,A> as core::ops::function::FnOnce<Args>>::call_once
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/alloc/src/boxed.rs:1951:9
   7: std::sys::unix::thread::Thread::new::thread_start
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std

[(1, Err(Any { .. })), (2, Ok(())), (3, Ok(())), (4, Err(Any { .. })), (5, Ok(())), (6, Err(Any { .. })), (7, Ok(())), (8, Err(Any { .. })), (9, Err(Any { .. }))]

Panic messages you see above is jupyter trying to handle panics in each thread. The correct result is at the end.